# Final Project - Idaho Policy Institute 5
## Ryan Pacheco, Ashley Gilbert, Ben Whitehead

Our goal is to identify characteristics which make a city sustainable, then classify cities based on whether they are growing sustainably or not. We will be looking at the cities in Idaho, California, and New York (state) with a population over 50,000.

## Initial Setup
We will start bby loading all of our data sources:
- US Census Data
- American Community Survey Data
- Greenhouse Gas Data (procured by the EPA)

In [1]:
%pip install census us

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import os

from census import Census
from us import states

import plotly.graph_objects as go

In [3]:
states.ID.shapefile_urls('county')

'https://www2.census.gov/geo/tiger/TIGER2010/COUNTY/2010/tl_2010_16_county10.zip'

In [4]:
#load census data using API key
c = Census('fb97753783c42ae57fe1a640e38fe04e921e5d1a')

#American Community Survey Data for California
i = 0
acs_years_ca = []
for x in range(2012, 2018):
    acs_test = c.acs5.state_place(('NAME',
                                   'B01003_001E',
                                   'B00002_001E',
                                   'B09018_007E',
                                   'B01002_001E'), states.CA.fips, '*', year=x)
    acs_years_ca.append(pd.DataFrame.from_records(acs_test))
    print(x)
    acs_years_ca[i] = acs_years_ca[i].rename(columns={
        'NAME' : 'City_Name',
        'place': 'FIPS',
        'B01003_001E': 'Total_Population_{}'.format(x),
        'B00002_001E': 'Total_Housing_{}'.format(x),
        'B09018_007E': 'Presence_of_Non-Relatives_{}'.format(x),
        'B01002_001E': 'Median_Age_{}'.format(x),
    })
    acs_years_ca[i].set_index('FIPS', inplace=True)
    acs_years_ca[i].drop(columns=['City_Name', 'state'], inplace=True)
    acs_years_ca[i] = acs_years_ca[i].nlargest(5, 'Total_Population_{}'.format(x))
    i = i + 1


#American Community Survey Data for New York
i = 0
acs_years_ny = []
for x in range(2012, 2018):
    acs_test = c.acs5.state_place(('NAME',
                                   'B01003_001E',
                                   'B00002_001E',
                                   'B09018_007E',
                                   'B01002_001E'), states.NY.fips, '*', year=x)
    acs_years_ny.append(pd.DataFrame.from_records(acs_test))
    print(x)
    acs_years_ny[i] = acs_years_ny[i].rename(columns={
        'NAME' : 'City_Name',
        'place': 'FIPS',
        'B01003_001E': 'Total_Population_{}'.format(x),
        'B00002_001E': 'Total_Housing_{}'.format(x),
        'B09018_007E': 'Presence_of_Non-Relatives_{}'.format(x),
        'B01002_001E': 'Median_Age_{}'.format(x),
    })
    acs_years_ny[i].set_index('FIPS', inplace=True)
    acs_years_ny[i].drop(columns=['City_Name', 'state'], inplace=True)
    acs_years_ny[i] = acs_years_ny[i].nlargest(5, 'Total_Population_{}'.format(x))
    i = i + 1


#American Community Survey Data for Idaho
i = 0
acs_years_id = []
for x in range(2012, 2018):
    acs_test = c.acs5.state_place(('NAME',
                                   'B01003_001E',
                                   'B00002_001E',
                                   'B09018_007E',
                                   'B01002_001E'), states.ID.fips, '*', year=x)
    acs_years_id.append(pd.DataFrame.from_records(acs_test))
    print(x)
    acs_years_id[i] = acs_years_id[i].rename(columns={
        'NAME' : 'City_Name',
        'place': 'FIPS',
        'B01003_001E': 'Total_Population_{}'.format(x),
        'B00002_001E': 'Total_Housing_{}'.format(x),
        'B09018_007E': 'Presence_of_Non-Relatives_{}'.format(x),
        'B01002_001E': 'Median_Age_{}'.format(x),
    })
    acs_years_id[i].set_index('FIPS', inplace=True)
    acs_years_id[i].drop(columns=['City_Name', 'state'], inplace=True)
    acs_years_id[i] = acs_years_id[i].nlargest(5, 'Total_Population_{}'.format(x))
    i = i + 1


#Greenhouse Gas Data
ghg = pd.DataFrame()

for f in os.listdir('data/2018_data_summary_spreadsheets'):
    temp = pd.read_excel('data/2018_data_summary_spreadsheets/'+f, sheet_name=0)
    temp['Year'] = f.split('.')[0].split('_')[2]    
    ghg = pd.concat([temp, ghg])
    
fips_map = pd.read_excel('data/fips-codes.xls', sheet_name=0)

fips_map = fips_map[fips_map['Entity Description'] == 'city']

def str_func(x):
    return str(x).zfill(5)

fips_map['FIPS Entity Code'] = fips_map['FIPS Entity Code'].apply(str_func)
fips_map['City'] = fips_map['GU Name']
fips_map['State'] = fips_map['State Abbreviation']

ghg_mapped = pd.merge(ghg, fips_map, on=['State', 'City'])

2012
2013
2014
2015
2016
2017
2012
2013
2014
2015
2016
2017
2012
2013
2014
2015
2016
2017


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:85: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




Now that all of our data is loaded, we will work on putting it all together for analysis

In [5]:
#common variables used for working with the census data.
keys = ['NAME' ,'P002001','P002002','P002005','H001001','P013001','H003001','P027001','H005001','H005002','H005003','H005004','H005005','H005006','H005007']

renames_2000 = {
        'NAME' : 'City_Name',
        'place': 'FIPS',
        'P002001': 'Total_Population_2000',
        'P002002':'Total_Urban_Population_2000',
        'P002005':'Total_Rural_Population_2000',
        'H001001': 'Total_Housing_2000',
        'P013001': 'Median_Age_2000',
        'H003001': 'Occupancy_Status_For_Housing_Units_2000',
        'P027001': 'Presence_of_Non-Relatives_2000',
        'H005001': 'Vacancy_Status_2000',
        'H005002': 'For_Rent_2000',
        'H005003': 'Rented_Not_Occupied_2000',
        'H005004': 'For_Sale_Only_2000',
        'H005005': 'Sold_Not_Occupied_2000',
        'H005006': 'For_Seasonal_Recreational_Or_Occasional_Use_2000',
        'H005007': 'For_Migrant_Workers_2000'
}

renames_2010 = {
        'NAME' : 'City_Name',
        'place': 'FIPS',
        'P002001': 'Total_Population_2010',
        'P002002':'Total_Urban_Population_2010',
        'P002005':'Total_Rural_Population_2010',
        'H001001': 'Total_Housing_2010',
        'P013001': 'Median_Age_2010',
        'H003001': 'Occupancy_Status_For_Housing_Units_2010',
        'P027001': 'Presence_of_Non-Relatives_2010',
        'H005001': 'Vacancy_Status_2010',
        'H005002': 'For_Rent_2010',
        'H005003': 'Rented_Not_Occupied_2010',
        'H005004': 'For_Sale_Only_2010',
        'H005005': 'Sold_Not_Occupied_2010',
        'H005006': 'For_Seasonal_Recreational_Or_Occasional_Use_2010',
        'H005007': 'For_Migrant_Workers_2010'}


## Merge data and start analysis

### California

In [6]:
city_2010 = c.sf1.state_place(keys, 
                              states.CA.fips, '*', year=2010)
c_pop_2010 = pd.DataFrame.from_records(city_2010)
c_pop_2010_50000 = c_pop_2010.rename(columns=renames_2010)

In [7]:
c_pop_2010_50000.head()

,Total_Housing_2010,Occupancy_Status_For_Housing_Units_2010,Vacancy_Status_2010,For_Rent_2010,Rented_Not_Occupied_2010,For_Sale_Only_2010,Sold_Not_Occupied_2010,For_Seasonal_Recreational_Or_Occasional_Use_2010,For_Migrant_Workers_2010,City_Name,Total_Population_2010,Total_Urban_Population_2010,Total_Rural_Population_2010,Median_Age_2010,Presence_of_Non-Relatives_2010,FIPS,state
0,457.0,457.0,16.0,4.0,1.0,1.0,2.0,2.0,0.0,"Acalanes Ridge CDP, California",1137.0,1137.0,0.0,46.3,441.0,00135,06
1,99.0,99.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,"Acampo CDP, California",341.0,0.0,341.0,30.6,94.0,00156,06
2,2814.0,2814.0,154.0,22.0,1.0,42.0,9.0,34.0,0.0,"Acton CDP, California",7596.0,0.0,7596.0,45.5,2660.0,00212,06
3,9086.0,9086.0,1277.0,462.0,11.0,323.0,56.0,32.0,0.0,"Adelanto city, California",31765.0,31381.0,384.0,25.3,7809.0,00296,06
4,144.0,144.0,20.0,1.0,0.0,1.0,0.0,9.0,0.0,"Adin CDP, California",272.0,0.0,272.0,47.3,124.0,00310,06


In [8]:
ghg_data['City_Name'] = ghg_data['City'] + ',' + ghg_data['State']
ghg_data.head()

NameError: name 'ghg_data' is not defined

In [ ]:
city_2000 = c.sf1.state_place(keys, states.CA.fips, '*', year=2000)
c_pop_2000 = pd.DataFrame.from_records(city_2000)
c_pop_2000_50000 = c_pop_2000.rename(columns=renames_2000)

In [ ]:
c_pop_2000_50000.drop(columns=['City_Name', 'state'], inplace=True)

In [ ]:
c_pop_2000_50000.head()

In [ ]:
c_pop_2000_50000.set_index('FIPS', inplace=True)
c_pop_2010_50000.set_index('FIPS', inplace=True)

In [ ]:
ca_join = c_pop_2000_50000.join(c_pop_2010_50000, on='FIPS')

In [ ]:
ca_join.head()

In [ ]:
ca_join['Total_Population_2000'] = ca_join['Total_Population_2000'].astype('i8')

In [ ]:
ca_join = ca_join.nlargest(5, 'Total_Population_2000')

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='2000_pop', x=ca_join['City_Name'], y=ca_join['Total_Population_2000']),
    go.Bar(name='2010_pop', x=ca_join['City_Name'], y=ca_join['Total_Population_2010']),
    go.Bar(name='2000_housing', x=ca_join['City_Name'], y=ca_join['Total_Housing_2000']),
    go.Bar(name='2010_housing', x=ca_join['City_Name'], y=ca_join['Total_Housing_2010']),
    go.Bar(name='2000_non-relatives', x=ca_join['City_Name'], y=ca_join['Presence_of_Non-Relatives_2000']),
    go.Bar(name='2010_non-relatives', x=ca_join['City_Name'], y=ca_join['Presence_of_Non-Relatives_2010']),
])
fig.update_layout(barmode='group')
fig.show()

## Get's the 5 largest cities in New York

In [ ]:
city_2010 = c.sf1.state_place(keys, states.NY.fips, '*', year=2010)
c_pop_2010 = pd.DataFrame.from_records(city_2010)
c_pop_2010_50000 = c_pop_2010.rename(columns=renames_2010)

In [ ]:
c_pop_2010_50000.head()

In [ ]:
city_2000 = c.sf1.state_place(keys, states.NY.fips, '*', year=2000)
c_pop_2000 = pd.DataFrame.from_records(city_2000)
c_pop_2000_50000 = c_pop_2000.rename(columns=renames_2000)

In [ ]:
c_pop_2000_50000.drop(columns=['City_Name', 'state'], inplace=True)

In [ ]:
c_pop_2000_50000.head()

In [ ]:
c_pop_2000_50000.set_index('FIPS', inplace=True)
c_pop_2010_50000.set_index('FIPS', inplace=True)

In [ ]:
ny_join = c_pop_2000_50000.join(c_pop_2010_50000, on='FIPS')

In [ ]:
ny_join.head()

In [ ]:
ny_join['Total_Population_2000'] = ny_join['Total_Population_2000'].astype('i8')

In [ ]:
ny_join = ny_join.nlargest(5, 'Total_Population_2000')

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='2000_pop', x=ny_join['City_Name'], y=ny_join['Total_Population_2000']),
    go.Bar(name='2010_pop', x=ny_join['City_Name'], y=ny_join['Total_Population_2010']),
    go.Bar(name='2000_housing', x=ny_join['City_Name'], y=ny_join['Total_Housing_2000']),
    go.Bar(name='2010_housing', x=ny_join['City_Name'], y=ny_join['Total_Housing_2010']),
    go.Bar(name='2000_non-relatives', x=ny_join['City_Name'], y=ny_join['Presence_of_Non-Relatives_2000']),
    go.Bar(name='2010_non-relatives', x=ny_join['City_Name'], y=ny_join['Presence_of_Non-Relatives_2010']),
])
fig.update_layout(barmode='group')
fig.show()

## Get's the 5 largest cities in Idaho

In [ ]:
city_2010 = c.sf1.state_place(keys, states.ID.fips, '*', year=2010)
c_pop_2010 = pd.DataFrame.from_records(city_2010)
c_pop_2010_50000 = c_pop_2010.rename(columns=renames_2010)

In [ ]:
c_pop_2010_50000.head()

In [ ]:
city_2000 = c.sf1.state_place(keys, states.ID.fips, '*', year=2000)
c_pop_2000 = pd.DataFrame.from_records(city_2000)
c_pop_2000_50000 = c_pop_2000.rename(columns=renames_2000)

In [ ]:
c_pop_2000_50000.drop(columns=['City_Name', 'state'], inplace=True)

In [ ]:
c_pop_2000_50000.head()

In [ ]:
c_pop_2000_50000.set_index('FIPS', inplace=True)
c_pop_2010_50000.set_index('FIPS', inplace=True)

In [ ]:
id_join = c_pop_2000_50000.join(c_pop_2010_50000, on='FIPS')

In [ ]:
id_join.head()

In [ ]:
id_join['Total_Population_2000'] = id_join['Total_Population_2000'].astype('i8')

In [ ]:
id_join =  id_join.nlargest(5, 'Total_Population_2000')

In [ ]:
id_join.head()

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='2000_pop', x=id_join['City_Name'], y=id_join['Total_Population_2000']),
    go.Bar(name='2010_pop', x=id_join['City_Name'], y=id_join['Total_Population_2010']),
    go.Bar(name='2000_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2000']),
    go.Bar(name='2010_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2010']),
    go.Bar(name='2000_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2000']),
    go.Bar(name='2010_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2010']),
])
fig.update_layout(barmode='group')
fig.show()

In [ ]:
three_state_df = pd.concat([id_join, ca_join, ny_join])

In [ ]:
three_state_df.reset_index(inplace=True)

In [ ]:
three_state_df.head()

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='2000_pop', x=three_state_df['City_Name'], y=three_state_df['Total_Population_2000']),
    go.Bar(name='2010_pop', x=three_state_df['City_Name'], y=three_state_df['Total_Population_2010']),
    go.Bar(name='2000_housing', x=three_state_df['City_Name'], y=three_state_df['Total_Housing_2000']),
    go.Bar(name='2010_housing', x=three_state_df['City_Name'], y=three_state_df['Total_Housing_2010']),
    go.Bar(name='2000_non-relatives', x=three_state_df['City_Name'], y=three_state_df['Presence_of_Non-Relatives_2000']),
    go.Bar(name='2010_non-relatives', x=three_state_df['City_Name'], y=three_state_df['Presence_of_Non-Relatives_2010']),
])
fig.update_layout(barmode='group')
fig.show()

* This graph is hard to gather any useful data from due to how New York City and Los Angeles are skewing the graph, let's drop those cities from the graph

In [ ]:
three_state_df.drop(three_state_df[three_state_df['City_Name'] =='Los Angeles city, California'].index, inplace = True)
three_state_df.drop(three_state_df[three_state_df['City_Name'] =='New York city, New York'].index, inplace = True)

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='2000_pop', x=three_state_df['City_Name'], y=three_state_df['Total_Population_2000']),
    go.Bar(name='2010_pop', x=three_state_df['City_Name'], y=three_state_df['Total_Population_2010']),
    go.Bar(name='2000_housing', x=three_state_df['City_Name'], y=three_state_df['Total_Housing_2000']),
    go.Bar(name='2010_housing', x=three_state_df['City_Name'], y=three_state_df['Total_Housing_2010']),
    go.Bar(name='2000_non-relatives', x=three_state_df['City_Name'], y=three_state_df['Presence_of_Non-Relatives_2000']),
    go.Bar(name='2010_non-relatives', x=three_state_df['City_Name'], y=three_state_df['Presence_of_Non-Relatives_2010']),
])
fig.update_layout(barmode='group')
fig.show()

* California is still being an issue, lets drop those cities form our graph

In [ ]:
three_state_df.drop(three_state_df[three_state_df['state'] ==states.CA.fips].index, inplace = True)

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='2000_pop', x=three_state_df['City_Name'], y=three_state_df['Total_Population_2000']),
    go.Bar(name='2010_pop', x=three_state_df['City_Name'], y=three_state_df['Total_Population_2010']),
    go.Bar(name='2000_housing', x=three_state_df['City_Name'], y=three_state_df['Total_Housing_2000']),
    go.Bar(name='2010_housing', x=three_state_df['City_Name'], y=three_state_df['Total_Housing_2010']),
    go.Bar(name='2000_non-relatives', x=three_state_df['City_Name'], y=three_state_df['Presence_of_Non-Relatives_2000']),
    go.Bar(name='2010_non-relatives', x=three_state_df['City_Name'], y=three_state_df['Presence_of_Non-Relatives_2010']),
])
fig.update_layout(barmode='group')
fig.show()

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='2000_age', x=three_state_df['City_Name'], y=three_state_df['Median_Age_2000']),
    go.Bar(name='2010_age', x=three_state_df['City_Name'], y=three_state_df['Median_Age_2010']),
])
fig.update_layout(barmode='group')
fig.show()

## American Community Survey

In [ ]:
i = 0
three_state_acs = []
for x in acs_years_ca:
    acs_1 = pd.concat([acs_years_ca[i], acs_years_ny[i], acs_years_id[i]])
    three_state_acs.append(acs_1)
    i = i + 1

In [ ]:
three_state_df.set_index('FIPS', inplace=True)

In [ ]:
for x in three_state_acs:
    print(x)
    three_state_df = three_state_df.join(x, on="FIPS")

In [ ]:
three_state_df.head()

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='2000_pop', x=three_state_df['City_Name'], y=three_state_df['Total_Population_2000']),
    go.Bar(name='2010_pop', x=three_state_df['City_Name'], y=three_state_df['Total_Population_2010']),
    go.Bar(name='2012_pop', x=three_state_df['City_Name'], y=three_state_df['Total_Population_2012']),
    go.Bar(name='2013_pop', x=three_state_df['City_Name'], y=three_state_df['Total_Population_2013']),
    go.Bar(name='2014_pop', x=three_state_df['City_Name'], y=three_state_df['Total_Population_2014']),
    go.Bar(name='2015_pop', x=three_state_df['City_Name'], y=three_state_df['Total_Population_2015']),
    go.Bar(name='2016_pop', x=three_state_df['City_Name'], y=three_state_df['Total_Population_2016']),
    go.Bar(name='2017_pop', x=three_state_df['City_Name'], y=three_state_df['Total_Population_2017']),
    go.Bar(name='2000_housing', x=three_state_df['City_Name'], y=three_state_df['Total_Housing_2000']),
    go.Bar(name='2010_housing', x=three_state_df['City_Name'], y=three_state_df['Total_Housing_2010']),
    go.Bar(name='2012_housing', x=three_state_df['City_Name'], y=three_state_df['Total_Housing_2012']),
    go.Bar(name='2013_housing', x=three_state_df['City_Name'], y=three_state_df['Total_Housing_2013']),
    go.Bar(name='2014_housing', x=three_state_df['City_Name'], y=three_state_df['Total_Housing_2014']),
    go.Bar(name='2015_housing', x=three_state_df['City_Name'], y=three_state_df['Total_Housing_2015']),
    go.Bar(name='2016_housing', x=three_state_df['City_Name'], y=three_state_df['Total_Housing_2016']),
    go.Bar(name='2017_housing', x=three_state_df['City_Name'], y=three_state_df['Total_Housing_2017']),
    go.Bar(name='2000_non-relatives', x=three_state_df['City_Name'], y=three_state_df['Presence_of_Non-Relatives_2000']),
    go.Bar(name='2010_non-relatives', x=three_state_df['City_Name'], y=three_state_df['Presence_of_Non-Relatives_2010']),
    go.Bar(name='2012_non-relatives', x=three_state_df['City_Name'], y=three_state_df['Presence_of_Non-Relatives_2012']),
    go.Bar(name='2013_non-relatives', x=three_state_df['City_Name'], y=three_state_df['Presence_of_Non-Relatives_2013']),
    go.Bar(name='2014_non-relatives', x=three_state_df['City_Name'], y=three_state_df['Presence_of_Non-Relatives_2014']),
    go.Bar(name='2015_non-relatives', x=three_state_df['City_Name'], y=three_state_df['Presence_of_Non-Relatives_2015']),
    go.Bar(name='2016_non-relatives', x=three_state_df['City_Name'], y=three_state_df['Presence_of_Non-Relatives_2016']),
    go.Bar(name='2017_non-relatives', x=three_state_df['City_Name'], y=three_state_df['Presence_of_Non-Relatives_2017']),
])
fig.update_layout(barmode='group')
fig.show()

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn.metrics
import sys
from pandas_ml import ConfusionMatrix

In [ ]:
mod = smf.glm('Total_Population_2000 ~ Total_Population_2010 + Total_Population_2012 + Total_Population_2013', three_state_df, family=sm.families.Binomial()).fit()
mod.summary()